# Testing integration with `vegas`

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ October 11, 2022

Trying to do a spectroscopic overlap calculation evaluating the integrals in the $\delta U^\dagger$ term with Monte Carlo integration using `vegas`. Currently attempting to do all the sums in the expression by looping (excluding as many terms equal to zero as possible), and use MC only for integration over momenta.

_Last update:_ October 25, 2022

In [1]:
import numpy as np
import numpy.linalg as la
from sympy.physics.quantum.cg import CG
import vegas

Testing `vegas` with example from tutorial [online](https://vegas.readthedocs.io/en/latest/tutorial.html)

In [2]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [3]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.62(30)        0.62(30)            0.00     1.00
  2   1.02(15)        0.94(13)            1.48     0.22
  3   1.00(18)        0.96(11)            0.78     0.46
  4   1.024(61)       1.008(53)           0.61     0.61
  5   0.937(35)       0.959(30)           0.76     0.55
  6   1.007(25)       0.987(19)           0.91     0.47
  7   1.019(19)       1.003(13)           1.00     0.42
  8   1.051(14)       1.0259(97)          1.73     0.10
  9   1.025(12)       1.0255(76)          1.52     0.15
 10   0.985(12)       1.0140(64)          2.26     0.02

result = 1.0140(64)    Q = 0.02


Testing integral from degenerate electron gas in Fetter and Walecka section 1.3

In [4]:
def heaviside(x):
    if x >= 0:
        return 1
    else:
        return 0

In [5]:
def g(Q_vector, kF=1):
    q_vector = np.repeat(kF/np.sqrt(3),3)
    integrand = (heaviside(kF-la.norm(Q_vector+q_vector/2))
                 * heaviside(kF-la.norm(Q_vector-q_vector/2)))
                    
    return integrand

In [8]:
integ = vegas.Integrator([[-3,3],[-3,3],[-3,3]])

mc_func = lambda Q_vector : g(Q_vector, kF=1.0)
result = integ(mc_func, nitn=20, neval=10000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))
print(f'True result = {5*np.pi/12}')

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.27(11)        1.27(11)            0.00     1.00
  2   1.07(14)        1.197(89)           1.17     0.28
  3   1.29(11)        1.236(69)           0.82     0.44
  4   1.454(87)       1.321(54)           1.85     0.14
  5   1.262(39)       1.282(31)           1.58     0.18
  6   1.315(26)       1.302(20)           1.39     0.22
  7   1.298(18)       1.300(13)           1.16     0.32
  8   1.322(23)       1.305(11)           1.10     0.36
  9   1.309(15)       1.3066(91)          0.97     0.46
 10   1.334(18)       1.3125(81)          1.08     0.37
 11   1.323(12)       1.3159(68)          1.03     0.42
 12   1.321(10)       1.3174(56)          0.95     0.49
 13   1.2845(88)      1.3079(47)          1.69     0.06
 14   1.2874(78)      1.3023(40)          1.95     0.02
 15   1.3048(74)      1.3029(36)          1.82     0.03
 16   1.47(17)        1.3030(35)          1.76  

In [9]:
print(result)
print(result.mean)
print(result.sdev)

1.2947(24)
1.2946725794990612
0.0023672833821018935
